Written by L. Robert

In [169]:
import sympy
from scipy import stats, optimize, integrate
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt

import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout

from IPython.display import display, Markdown
  
from ipywidgets import interact
import ipywidgets as widgets
import sympy.printing as printing
from IPython.display import display, Math, Latex
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline
from sympy import symbols, Eq, solve

#### Definition of Parameters

In [170]:
#Definition of symbols for parameters. 
n=sympy.symbols('n')
n_j=sympy.symbols('n_j')
n_A=sympy.symbols('n_A')
n_j_A=sympy.symbols('n^j_A')
n_B=sympy.symbols('n_B')
n_j_B=sympy.symbols('n^j_B')
q_A=sympy.symbols('q_A')
q_B=sympy.symbols('q_B')
q_i=sympy.symbols('q_i')
Q_OSS=sympy.symbols('Q^OSS')
Q_CSS=sympy.symbols('Q^CSS')
Q_j=sympy.symbols('Q^j')
k_A=sympy.symbols('k_A')
k_B=sympy.symbols('k_B')
v_A=sympy.symbols('v_A')
v_B=sympy.symbols('v_B')
alpha=sympy.symbols('α')
beta=sympy.symbols('β')
sum=sympy.symbols('sum')
gleich=sympy.symbols('=')

# Jupyter Notebook Analysis 

## Description

Within the scope of my bachelor thesis I analyzed the paper: “Open Source as a Signaling Device - An Economic Analysis" by Samuel Lee, Nina Moisa and Marco Weiß (2003). In their work they analyze under which circumstances programmers contribute to Open Source Software (OSS).
Their approach is that programmers contributes to OSS because they want to show off their programming skills. This way they can signal to a company how good they are at programming. If the company hires them, the programmers will get paid adequately to their productivity. The better the programming skills, the higher their productivity. 
Given this general idea, they wanted to explore the different scenarios when there are two types of programmers, how the programmers would act on the market if there was a OSS and a closed source software (CSS). One type of programmer is less productive than the other.

Using the tools available to me on Jupyter Notebook, I want to further analyze the model in paper by Lee et. al. (2003) and show their results in a different light. I will take their work a little further, especially in the case of graphs which have dynamic variables.

## Analysis of the Paper

There is a total population of $n$, in which there are programmers of type A and programmers of type B.

In [171]:
display(Math('n=n_A+n_B'))

<IPython.core.display.Math object>

The different types have different productivity $q$ and therefore different output. The amount of output is also defined as the quality of the programmer’s  work.

In [172]:
x = (q_A > q_B )
x

q_A > q_B

The production function defines the total output. In this is case human capital the only input for the production.

In [173]:
display(Math('Q_j=\sum \limits _{i=1} ^{n_j} q_{i} = n^{j}_{A} q_{A} + n^{j}_{B} q_{B}'))
display(Math('for j=OSS, CSS'))
display(Math('n^{j}_{A} \ and \ n^{j}_{B} \ are \ the \ amount \ of \ programmers\ of  \ type \ A \ or \ B  \ in \ the \ two \ institutions.'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

 $α$ and $β$ define the percentage of each type of programmer in the CSS over the total population. $α$ stand for the percentage of $n_A$ in the CSS, $β$ for the percentage of $n_B$ in the CSS. The rest of the population is part of the OSS community.

In [174]:
display(Math('0 < α < 1'))
display(Math('0 < β < 1'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The total amount of output from the OSS is defined by:

In [175]:
display(Math('Q^{OSS}= (1-α) n^{j}_{A} q_{A}+ (1-β) n^{j}_{B} q_{B}'))

<IPython.core.display.Math object>

In [176]:
def output_OSS(alpha, beta):
    return (1-alpha)*n_A*q_A+(1-beta)*n_B*q_B

This is the total amount of output from the CSS:

In [177]:
display(Math('Q^{CSS}= α n^{j}_{A} q_{A}+ β n^{j}_{B} q_{B}'))

<IPython.core.display.Math object>

In [178]:
def output_CSS(alpha, beta):
    return alpha*n_A*q_A+beta*n_B*q_B

The wage in the CSS gets calculated by the amount of output divided by the number of workers in the CSS. Every employee gets the same wage since the company cannot distinguish between the two types. We assume, that the CSS makes zero profits. 

In [179]:
display(Math(rf'w(α, β)= \frac{{Q^\text{{CSS}}}}{{n^\text{{CSS}}}}\, = \frac{{αn_A q_A + β n_B q_B}}{{α n_A + β n_B}}'))

<IPython.core.display.Math object>

In [180]:
def wage(alpha, beta):
    return (alpha*n_A*q_A+beta*n_B*q_B)/(alpha*n_A+beta*n_B)

The following graph will show high the wage for programmers in the CSS is:

In [181]:
from mpl_toolkits.mplot3d import Axes3D
def plot(n_A, n_B, q_A, q_B):
    fig = plt.figure()
    ax = fig.add_subplot(111,projection='3d')
    alpha = np.arange(0.001, 1, 0.1)
    beta = np.arange(0.001, 1, 0.1)
    alpha, beta = np.meshgrid(beta, alpha)
    wage = ((alpha*n_A*q_A)+(beta*n_B*q_B))/((alpha*n_A)+(beta*n_B)) 
   
    ax.plot_surface(beta, alpha, wage, linewidth=0.25)
    ax.set_xlabel('β') 
    ax.set_ylabel('α') 
    ax.set_zlabel('wage')
    plt.show()
    
interact(plot, n_A = (1, 100, 1), n_B = (1, 100, 1), q_A = (1, 100, 1), q_B = (1, 100, 1))





interactive(children=(IntSlider(value=50, description='n_A', min=1), IntSlider(value=50, description='n_B', mi…

<function __main__.plot(n_A, n_B, q_A, q_B)>

Here the specific marginal effects on the wage the parameters $n_A, n_B, q_A$ and $q_B$ have. 

In [182]:
sympy.diff((alpha*n_A*q_A+beta*n_B*q_B)/(alpha*n_A+beta*n_B), n_A)

q_A*α/(n_A*α + n_B*β) - α*(n_A*q_A*α + n_B*q_B*β)/(n_A*α + n_B*β)**2

The value above will never be zero. The marginal effect of one more programmer of type A will always be positive.

In [183]:
sympy.diff((alpha*n_A*q_A+beta*n_B*q_B)/(alpha*n_A+beta*n_B), q_A)

n_A*α/(n_A*α + n_B*β)

The marginal effect on the wage of one more unit of productivity from a programmer of type A.

In [184]:
sympy.diff((alpha*n_A*q_A+beta*n_B*q_B)/(alpha*n_A+beta*n_B), n_B)

q_B*β/(n_A*α + n_B*β) - β*(n_A*q_A*α + n_B*q_B*β)/(n_A*α + n_B*β)**2

The value above will never be zero. The negative marginal effect of one more programmer of type B will always be negative.

In [185]:
sympy.diff((alpha*n_A*q_A+beta*n_B*q_B)/(alpha*n_A+beta*n_B), q_B)

n_B*β/(n_A*α + n_B*β)

The marginal effect on the wage of one more unit of productivity from a programmer of type B.

Additionally to the paper we see how the quality of the code and the amount of each type of programmer will influence the wage a company will pay to its employees.  

### Deferred Payoff

By contributing to OSS programmers may earn signals that indicate their productivity and programming skills. These can lead to a job-offer from another company, where they could get an adequate salary. This is the deferred payoff for the work they are doing right at that moment.  

The following functions represent the amount of deferred payoff a programmer could get:

In [186]:
display(Math(rf'r_A(α)=-\frac{{4 n_A^2 v_A}}{{k_A^2}}\ [α-(1- \frac{{k_A}}{{2 n_A}})]^2 + v_A' ))

<IPython.core.display.Math object>

In [187]:
def deferred_payoff_A(alpha):
    return -((4*(n_A)**2*v_A)/(k_A)**2)*(alpha-(1-(k_A)/(2*n_A)))**2+v_A  
deff_pay_A=deferred_payoff_A(alpha)

In [188]:
display(Math(rf'r_B(β)= -\frac{{4 n_B^2 v_B}}{{k_B^2}}\ [β-(1- \frac{{k_B}}{{2 n_B}})]^2 + v_B,' ))

<IPython.core.display.Math object>

In [189]:
def deferred_payoff_B(beta):
    return -((4*(n_B)**2*v_B)/(k_B)**2)*(beta-(1-(k_B)/(2*n_B)))**2+v_B
deff_pay_B=deferred_payoff_B(beta)    

The more contributors there are, the more programmers must excel to distinguish themself from other programmers in the OSS community. Therefore, the probability to get a signal is negatively correlated with the amount of contributors. But the more contributors there are with the OSS, the more contributions there will be made for the project and the higher the quality of the OSS is. 

The variable $k$ is the potential of innovation in the industry. This can be interpreted as the number of problems that can be solved by innovation. The higher the potential, the less competition there will be and the higher the visibility of the project. This leads to a higher probability that a programmer can earn a signal. In the functions bellow, $k$ defines the number of programmers of type A or B that will get zero payoff, depending on other parameters.

A company is interested in new talents that can be discovered in an OSS project. If they are risk-averse and fear asymmetric information on the market, they might pay a fee to be informed of the talents in the OSS community. But they will most probably only pay for information for programmers of type A. These elements are subsumed in the variables $v_A$ and $v_B$. It is related to the personal productivity.

The following graph will show high the differed payoff is for programmers in the OSS is. this It is dependent of $k_i$, $v_i$ and $n_i$. 

In [190]:
def plot(n_A, n_B, v_A, v_B, k_A, k_B):
    fig = plt.figure()
    fig, ax = plt.subplots()
    alpha = np.arange(0, 1, 0.01)
    beta = np.arange(0, 1, 0.01)
    
    pay_A= -((4*(n_A)**2*v_A)/(k_A)**2)*(alpha-(1-(k_A)/(2*n_A)))**2+v_A
    pay_B= -((4*(n_B)**2*v_B)/(k_B)**2)*(beta-(1-(k_B)/(2*n_B)))**2+v_B
    zero= alpha*0
   
    plt.plot(alpha, pay_A, 'blue')
    plt.plot(beta, pay_B, 'red')
    plt.plot(alpha, zero, 'black' )
    
    ax.set_xlabel('α, β') 
    ax.set_ylabel('$r_i$') 
    
    plt.show()
    
interact(plot, n_A = (1, 100, 1), n_B = (1, 100, 1), v_A = (1, 100, 1), v_B = (1, 100, 1), k_A = (1, 100, 1), k_B = (1, 100, 1) )


interactive(children=(IntSlider(value=50, description='n_A', min=1), IntSlider(value=50, description='n_B', mi…

<function __main__.plot(n_A, n_B, v_A, v_B, k_A, k_B)>

In [191]:
sympy.diff(deff_pay_A, n_A)

4*v_A*(k_A/(2*n_A) + α - 1)/k_A - 8*n_A*v_A*(k_A/(2*n_A) + α - 1)**2/k_A**2

The marginal effect on the deferred payoff of one more programmer of type A.

In [192]:
sympy.diff(deff_pay_A, v_A)

1 - 4*n_A**2*(k_A/(2*n_A) + α - 1)**2/k_A**2

The marginal effect on the deferred payoff of one more unit of $v_A$.

In [193]:
sympy.diff(deff_pay_A, k_A)

-4*n_A*v_A*(k_A/(2*n_A) + α - 1)/k_A**2 + 8*n_A**2*v_A*(k_A/(2*n_A) + α - 1)**2/k_A**3

The marginal effect on the deferred payoff of one more unit of $k_A$.

In [194]:
sympy.diff(deff_pay_B, n_B)

4*v_B*(k_B/(2*n_B) + β - 1)/k_B - 8*n_B*v_B*(k_B/(2*n_B) + β - 1)**2/k_B**2

The marginal effect on the deferred payoff of one more programmer of type B.

In [195]:
sympy.diff(deff_pay_B, v_B)

1 - 4*n_B**2*(k_B/(2*n_B) + β - 1)**2/k_B**2

The marginal effect on the deferred payoff of one more unit of $v_B$.

In [196]:
sympy.diff(deff_pay_B, k_B)

-4*n_B*v_B*(k_B/(2*n_B) + β - 1)/k_B**2 + 8*n_B**2*v_B*(k_B/(2*n_B) + β - 1)**2/k_B**3

The marginal effect on the deferred payoff of one more unit of $k_B$.

## Analysis of the Equilibrium

### Analyzing the decision of type B

From here on we assume that $\alpha$ and $\beta$ are known with certainty. We look at a static equilibrium which is a state where there is no tendency for change. An agent in this equilibrium will presume that his action will not affect the overall distribution and will therefor only take the wage and the delayed payoff as a reference for his actions. 

Difference of pay:

In [197]:
display(Math('Δ_A (α) = w(α, β)-r_A(α)'))
display(Math('Δ_B (β) = w(α, β)-r_A(β)'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [198]:
def difference_A(alpha, beta):
    return wage(alpha, beta)-deferred_payoff_A(alpha)
    
def difference_B(alpha, beta):
    return wage(alpha, beta)-deferred_payoff_B(beta)

$\Delta_i$ shows the decision each type of programmer will take. If the respective $\Delta_i$ is negative, they will begin program for the OSS, if it is positive, they will continue programming for the CSS.

In [199]:
difference_B(alpha, beta)

-v_B + (n_A*q_A*α + n_B*q_B*β)/(n_A*α + n_B*β) + 4*n_B**2*v_B*(k_B/(2*n_B) + β - 1)**2/k_B**2

In [200]:
display(Math('w(α, β) = r_A(α)'))
display(Math('w(α, β) = r_A(β)'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In this case both types of programmers are indifferent.

In [201]:
difference_B(alpha, beta)

-v_B + (n_A*q_A*α + n_B*q_B*β)/(n_A*α + n_B*β) + 4*n_B**2*v_B*(k_B/(2*n_B) + β - 1)**2/k_B**2

The difference-function can never be negative for type B since they will never eran more on the market than is equal to their productivity. So, a programmer of type B will always work for the CSS.

With this new information that $\beta = 1$ we have a new wage function for programmers of type A.

In [202]:
pay_A=wage(alpha, 1)
pay_A

(n_A*q_A*α + n_B*q_B)/(n_A*α + n_B)

The following graph will show high the new wage is for programmers of type A in the CSS is.

In [203]:
def plot(q_A, q_B, n_A, n_B):
    fig = plt.figure()
    fig, ax = plt.subplots()
    alpha = np.arange(0.01, 1, 0.01)
    
    
    pay_A= q_B+((n_A*(q_A-q_B))/(n_A+alpha*n_A))*alpha
    
    plt.plot(alpha, pay_A)
    
    
    ax.set_xlabel('α') # Add an x-label to the axes.
    ax.set_ylabel('w') # Add a y-label to the axes.
    
    plt.show()
    
interact(plot, q_A =(1, 100, 1), q_B=(1, 100, 1), n_A=(1, 100, 1), n_B=(1, 100, 1))


interactive(children=(IntSlider(value=50, description='q_A', min=1), IntSlider(value=50, description='q_B', mi…

<function __main__.plot(q_A, q_B, n_A, n_B)>

In [204]:
sympy.diff(pay_A, n_A)

q_A*α/(n_A*α + n_B) - α*(n_A*q_A*α + n_B*q_B)/(n_A*α + n_B)**2

The marginal effect on the wage of type A programmers of one more programmer of type A.

In [205]:
sympy.diff(pay_A, n_B)

q_B/(n_A*α + n_B) - (n_A*q_A*α + n_B*q_B)/(n_A*α + n_B)**2

The marginal effect on the wage of type A programmers of one more programmer of type B.

In [206]:
sympy.diff(pay_A, q_A)

n_A*α/(n_A*α + n_B)

The marginal effect on the wage of type A programmers of one more unit of $q_A$.

In [207]:
sympy.diff(pay_A, q_B)

n_B/(n_A*α + n_B)

The marginal effect on the wage of type A programmers of one more unit of $q_B$.

### Analyzing the decision of type A

If there were no programmers of type B, all programmers of type A would work for the CSS because the wage would be $q_A$. Since the overall quality of the code will be $q_A$ all programmers will be compensated with the wage of $q_A$. They have no initiative to go and work for the OSS, since they would not get better offers on the market. 

In [208]:
difference= q_A-deferred_payoff_A(alpha)
difference

q_A - v_A + 4*n_A**2*v_A*(k_A/(2*n_A) + α - 1)**2/k_A**2

If we combine the wage function and the function  for the deferred payoff to get $\Delta_A$, the function is:

In [209]:
difference_A(alpha, beta)

-v_A + (n_A*q_A*α + n_B*q_B*β)/(n_A*α + n_B*β) + 4*n_A**2*v_A*(k_A/(2*n_A) + α - 1)**2/k_A**2

The second addend will never be negative. For the first addends it is dependent on the relation between $v_A$ and the productivity of each type ($q_A$ and $q_B$).

If we put $\beta=1$ in the function above, as we have determined befor is the case on the market, we will get the following function

In [210]:
difference_A(alpha, 1)

-v_A + (n_A*q_A*α + n_B*q_B)/(n_A*α + n_B) + 4*n_A**2*v_A*(k_A/(2*n_A) + α - 1)**2/k_A**2

So, the first addend is the only that could be negative. That is the case if $v_A$ exceeds $q_B$ to a sufficient degree. Since $q_B$ is the lower bound for the wage function and $v_A$ is the maximum of the deferred payoff the condition $q_B < v_A$ is a minimal condition for the signaling mechanism. 

If we change the parameters of $k_A=2 n_A$ and $v_A = q_A$ in the deferred payoff function, we will get the following function:

In [211]:
def deferred_payoff_A(alpha):
    return -((4*(n_A)**2*q_A)/(2*n_A)**2)*(alpha-(1-(q_A)/(2*n_A)))**2+q_A

In [212]:
difference_A(alpha, 1)

q_A*(α - 1 + q_A/(2*n_A))**2 - q_A + (n_A*q_A*α + n_B*q_B)/(n_A*α + n_B)

If there are no programmers of type A in the CSS:

In [213]:
difference_A(0, 1)

q_A*(-1 + q_A/(2*n_A))**2 - q_A + q_B

This will be a result that is less than zero. So, there is a constellation for $q_B < v_A$ and $0 < n_B$ that will lead to a non-positive value. That is a sufficient condition for a type A programmer to join the OSS.  

From all the functions above, we can follow that a CSS exists in any case. From the condition just before we can deduce that under certain conditions it is possible that a OSS will co-exist. But it will not necessarily be established. For example, if $\alpha = 1$.

## Comparative statistics

The number of equilibria depends on the relation between the number of programmers of type A and the total number of programmers that will get a value of zero for the deferred payoff. We assume that $\beta=1$ and $q_A=v_A$, so programmers of type A will get a salary adequate to their skills. 
We distinguish three cases, that will be established in the following cells:

#### CASE 1

In [214]:
display(Math (rf"\frac{{n_A}}{{k_A}}\ < \frac{{1}}{{2}}\ (1-\sqrt{{1- \frac{{q_B}}{{q_A}} \ }}\ ) " ))  

<IPython.core.display.Math object>

In this case only the right-hand corner solution will hold for an equilibrium. For a certain innovative potential and a comparatively low number of type A programmer it is difficult to get a OSS started that is qualitatively good. In this case the OSS could not get the visibility and credibility that the programmers can get a signal for an adequate salary to their skills. In this equilibrium there will be no OSS, only the CSS exists.   

#### CASE 2

In [215]:
display(Math (rf" \frac{{1}}{{2}}\ (1-\sqrt{{1- \frac{{q_B}}{{q_A}} \ }}\ ) \leq  \frac{{n_A}}{{k_A}}\ < \frac{{1}}{{2}}\ (1+\sqrt{{1 - \frac{{q_B}}{{q_A}} \ }}\ ) " ))  

<IPython.core.display.Math object>

Here an OSS is possible. Since the number of type A programmers is relatively high compared to the innovative potential in the market the signal they could earn in the OSS would be above the wage they get if they would be working for the OSS. At the same time, the number of type A programmers is low enough compared to the total population so that their adequat salary is still above the wage level. 

#### CASE 3

In [216]:
display(Math (rf" \frac{{1}}{{2}}\ (1+\sqrt{{1- \frac{{q_B}}{{q_A}} \ }}\ ) \leq \frac{{n_A}}{{k_A}}\  " ))  

<IPython.core.display.Math object>

In this case the number of type A programmers is high enough that a qualitatively good OSS can come into existence from a certain $\alpha$ on. But also, if $\alpha$ is high enough the deferred payoff a programmer might get with the OSS is lower than the salary earned in the CSS. That means: If the transfer-system of information is well developed so that $v_A=q_A$ there can only be an OSS on the market, if the number of programmers of type A is large enough compared to the total population.  

These are all necessary conditions. We cannot conclude with the analysis if an OSS will emerge. 

There is a threshold for $\alpha$, so that from a certain point on it will not be possible for an OSS to exist since there are too many programmers of type A contributors in the OSS. The deferred payoff will be lower than the wage in the CSS. Or there are not enough programmer so that the OSS is not of good quality and programmers could not earn signals. There can be triggers that $\alpha$ goes below the threshold. The nearer $\alpha$ is to one, the higher the probability is for a trigger. The lower the potential for innovation, the lower the probability that the programmer in the OSS can earn signals. 

## Importance of information transfer

For the next analysis we assume that $k_A$ is equal to zero and $v_A$ is not equal to the productivity of programmer of type A. This leads to the following function for the deferred payoff:

In [217]:
def deff_pay(alpha):
    return -4*v_A* (alpha-1/2)**2 + v_A

deff_pay(alpha)

-4*v_A*(α - 0.5)**2 + v_A

In [218]:
deff_pay(0)

0

In [219]:
deff_pay(1)

0

The optimum would be:

In [220]:
deff_pay(0.5)

v_A

The co-existence of OSS and CSS is only possible if the environment on the market permits to transfer signals to the market. The more open the market is and the higher it's valuation, the more probable it is that there is a co-existence on the market. 

If $v_A$ is low, that means that the transfer mechanism for informations is non-existent or not developed well enough. With the Internet emerging the transfer of information gets better. This increases the probability of a OSS to emerge.  

## Strategic wage setting

Until now we assumed that the CSS sets the wage in a way that it makes zero profits. Here we assume that $k_A=n_A$ and that a firm can chose the employee’s wage. Still there is a pooling wage. The new wage is denoted with $\hat{w}$. There the minimum wage is $q_B$. If $\hat{w}$ surpasses $v_A$ there will be no OSS, only the CSS. 
If the wage gets lowered, the profit will increase. But that increases the probability of a duopoly in the market with an OSS. 

In [221]:
def new_wage(): 
    return -4*v_A*(alpha-(1/2))**2+v_A
new_wage()

-4*v_A*(α - 0.5)**2 + v_A

In [222]:
display(Math(rf' α_1 = \frac{{1}}{{2}}\ - \sqrt{{ \frac{{v_A- \hat{{w}}\ }}{{4v_A}}\ }} \ '))

<IPython.core.display.Math object>

In [223]:
display(Math(rf' α_1 = \frac{{1}}{{2}}\ + \sqrt{{ \frac{{v_A- \hat{{w}}\ }}{{4v_A}}\ }} \ '))

<IPython.core.display.Math object>

There is no solution for $\hat{w} > v_A$ as we see. The OSS will not existe in this case.

## Literature

Lee, S., Moisa, N., & Weiß, M. (2003). Open source as a signaling sevice - An economic analysis (No. 102) Working Paper Series: Finance & Accounting. https://www.econstor.eu/bitstream/10419/76971/1/wp102.pdf